In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import cv2
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
img_height = 300
img_width = 300
batch_size = 32

In [3]:
age_set = []
gender_set = []
race_set = []
face_img_pixels_set = []
path = "./FaceRecognitionDataset"
dir_list = os.listdir(path)

for img_dir in dir_list:
    try:
        img_path = "./FaceRecognitionDataset"
        img_path += "/" + img_dir
        face_img_dir = os.listdir(img_path)
        for face_img in face_img_dir:
            if ".jpg" in face_img:
                face_img_split = face_img.split("_")
                age_set_str_int = face_img_split[0]
                age_set_str_int = int(age_set_str_int)
                age_set.append(age_set_str_int)
                gender_set_str_int = face_img_split[1]
                gender_set_str_int = int(gender_set_str_int)
                gender_set.append(gender_set_str_int)
                race_set_str_int = face_img_split[2]
                race_set_str_int = int(race_set_str_int)
                race_set.append(race_set_str_int)
                face_img_path = img_path + "/" + face_img
                face_img_pixels = cv2.imread(face_img_path)
                dim = (img_height, img_width)
                img_resize = cv2.resize(face_img_pixels, dim)
                cv2.imwrite(face_img_path, img_resize) 
                face_img_pixels_resized = cv2.imread(face_img_path)
                face_img_pixels_set.append(face_img_pixels_resized)     
    except:
        pass

In [4]:
len(face_img_pixels_set)
face_img_pixels_set = np.stack(face_img_pixels_set)

In [5]:
print(face_img_pixels_set.shape)

(28663, 300, 300, 3)


In [6]:
race_set = np.array(race_set)

In [7]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

2021-11-05 18:44:45.840595: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
print()

In [9]:
pixel_data_train, pixel_data_test, race_data_train, race_data_test = train_test_split(face_img_pixels_set, race_set, 
                                                                                        test_size = 0.2, random_state=123)
pixel_data_train, pixel_data_val, race_data_train, race_data_val = train_test_split(pixel_data_train, race_data_train,
                                                                                     test_size = 0.25,  random_state=123)

In [10]:
normalization_layer = layers.Rescaling(1./255)

In [11]:
num_classes = 5

model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [12]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 300, 300, 3)       0         
_________________________________________________________________
rescaling_1 (Rescaling)      (None, 300, 300, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 300, 300, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 150, 150, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 64)       

In [ ]:
epochs = 30
print(pixel_data_train[2].shape)
print(pixel_data_train.shape, pixel_data_train.dtype)
X = tf.convert_to_tensor(pixel_data_train)
print(X.shape)
Y = tf.convert_to_tensor(race_data_train, dtype=tf.float32)
history = model.fit(
  x = X, y = Y,
  validation_data = (pixel_data_val, race_data_val),
  epochs=epochs
)

(300, 300, 3)
(17197, 300, 300, 3) uint8
(17197, 300, 300, 3)


2021-11-05 18:49:06.231758: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30
538/538 [==============================] - 2249s 4s/step - loss: 1.3975 - accuracy: 0.4585 - val_loss: 1.3100 - val_accuracy: 0.4697
Epoch 2/30
538/538 [==============================] - 779s 1s/step - loss: 1.3024 - accuracy: 0.4768 - val_loss: 1.2597 - val_accuracy: 0.4968
Epoch 3/30
448/538 [=======================>......] - ETA: 1:59 - loss: 1.2696 - accuracy: 0.4911

In [ ]:
print(tf.__version__)